In [44]:
import cv2
import numpy as np
import os
import csv
import matplotlib.pyplot as plt
import imageio
from PIL import Image, ImageDraw


In [45]:
#READ IMAGES FROM FOLDER
images = []
def read_all_images():
    
    folder=r'C:\Users\yigit\Desktop\stajdatades\uygun'
    for filename in os.listdir(folder):
        img=Image.open(os.path.join(folder,filename)).convert("RGB")
        #img = cv2.imread(os.path.join(folder,filename),cv2.IMREAD_GRAYSCALE)
        img=np.array(img)
 
        if img is not None:
            images.append(img)
            
    return images


In [46]:
#Extract the blood vessels from the images
def extract_bv(image):
    b,green_fundus,r = cv2.split(image)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    contrast_enhanced_green_fundus = clahe.apply(green_fundus)

    # applying alternate sequential filtering (3 times closing opening)
    r1 = cv2.morphologyEx(contrast_enhanced_green_fundus, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5)), iterations = 1)
    R1 = cv2.morphologyEx(r1, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5)), iterations = 1)
    r2 = cv2.morphologyEx(R1, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,11)), iterations = 1)
    R2 = cv2.morphologyEx(r2, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,11)), iterations = 1)
    r3 = cv2.morphologyEx(R2, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(23,23)), iterations = 1)
    R3 = cv2.morphologyEx(r3, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(23,23)), iterations = 1)	
    f4 = cv2.subtract(R3,contrast_enhanced_green_fundus)
    f5 = clahe.apply(f4)

    # removing very small contours through area parameter noise removal
    ret,f6 = cv2.threshold(f5,15,255,cv2.THRESH_BINARY)	
    mask = np.ones(f5.shape[:2], dtype="uint8") * 255	
    contours, hierarchy = cv2.findContours(f6.copy(),cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        if cv2.contourArea(cnt) <= 200:
            cv2.drawContours(mask, [cnt], -1, 0, -1)
    im = cv2.bitwise_and(f5, f5, mask=mask)
    ret,fin = cv2.threshold(im,15,255,cv2.THRESH_BINARY_INV)
    newfin = cv2.erode(fin, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3)), iterations=1)	

    # removing blobs of unwanted bigger chunks taking in consideration they are not straight lines like blood
    #vessels and also in an interval of area
    fundus_eroded = cv2.bitwise_not(newfin)	
    xmask = np.ones(fundus.shape[:2], dtype="uint8") * 255
    xcontours, xhierarchy = cv2.findContours(fundus_eroded.copy(),cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)	
    for cnt in xcontours:
        shape = "unidentified"
        peri = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.04 * peri, False)   
        if len(approx) > 4 and cv2.contourArea(cnt) <= 3000 and cv2.contourArea(cnt) >= 100:
            shape = "circle"	
        else:
            shape = "veins"
        if(shape=="circle"):
            cv2.drawContours(xmask, [cnt], -1, 0, -1)

    finimage = cv2.bitwise_and(fundus_eroded,fundus_eroded,mask=xmask)	
    blood_vessels = cv2.bitwise_not(finimage)
    return blood_vessels

#FOR TAGGING
def blurry_or_not(laplacian_value):
    
    if(laplacian_value<=3200):
        return 'Bulanik'
    elif(laplacian_value<=5000):
        return 'Bulanik Olabilir'
    else:
        return 'Uygun'
    
    


In [47]:
read_all_images()
values=[]
total=0
counter=0
#CALL EXTARCTING IMAGE AND TAKE LAPLACIAN ONE BY ONE
for i in range(len(images)):
    img=extract_bv(images[i])
    hs=cv2.Laplacian(img, cv2.CV_64F).var()
    total=hs+total
    values.append(hs)
    counter=counter+1
    print(counter,blurry_or_not(hs))
print('ORT',total/len(values))
print('MAX',np.amax(values))
print('MİN', np.amin(values))


1 Uygun
2 Uygun
3 Uygun
4 Bulanik Olabilir
5 Uygun
6 Uygun
7 Uygun
8 Uygun
9 Uygun
10 Uygun
11 Uygun
12 Uygun
13 Uygun
14 Uygun
15 Uygun
16 Uygun
17 Bulanik Olabilir
18 Bulanik Olabilir
19 Uygun
20 Uygun
21 Uygun
22 Uygun
23 Uygun
24 Uygun
25 Uygun
26 Uygun
27 Uygun
28 Uygun
29 Bulanik Olabilir
30 Bulanik Olabilir
31 Uygun
32 Uygun
33 Uygun
34 Uygun
35 Uygun
36 Uygun
37 Uygun
38 Uygun
39 Uygun
40 Uygun
41 Uygun
42 Uygun
43 Uygun
44 Uygun
45 Uygun
46 Uygun
47 Uygun
48 Uygun
49 Uygun
50 Uygun
51 Uygun
52 Uygun
53 Uygun
54 Uygun
55 Uygun
56 Uygun
57 Uygun
58 Uygun
59 Uygun
60 Uygun
61 Uygun
62 Uygun
63 Uygun
64 Uygun
65 Uygun
66 Uygun
67 Uygun
68 Uygun
69 Uygun
70 Uygun
71 Uygun
72 Uygun
73 Uygun
74 Uygun
75 Uygun
76 Uygun
77 Uygun
78 Bulanik Olabilir
79 Uygun
80 Uygun
81 Uygun
ORT 7268.839895813701
MAX 10625.251292236124
MİN 3230.860233070322
